In [1]:
import numpy as np
import os
import cv2
import scipy.io as sio
from densenet121 import DenseNet
import tensorflow as tf
import time

from keras.optimizers import SGD, Adagrad, Adadelta, RMSprop, Adam, Nadam
import keras.backend as K
import tensorflow as tf
import scipy.io as sio
from keras.models import Model
from keras.regularizers import l2, l1
# from densenet121 import DenseNet
# from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Input, Concatenate, LeakyReLU, UpSampling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import RMSprop, Adam, Adadelta
from keras.callbacks import Callback, ModelCheckpoint
from keras.models import load_model

from keras.models import Sequential
from keras.utils import plot_model
import keras as k
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D,Activation,Input, Reshape, Permute, GlobalAveragePooling2D
from keras import optimizers
import numpy as np
# import matplotlib.pyplot as plt
from scipy.misc import imresize, imsave
import csv
import scipy.io as sio
from PIL import Image
import scipy
import pickle
from scipy.misc import imresize


Using TensorFlow backend.


In [2]:
def create_dir(dirName):
    if not os.path.exists(dirName):
        os.mkdir(dirName)

In [3]:
#Set root path 
Network = 'deepCount/'
root_pth ='/media/akhtar/6D2C8F896B2F79E0/naseer_houseCounting/fusionNet/' +  Network
#set path of dataset
dataset_path = '/media/akhtar/6D2C8F896B2F79E0/naseer_houseCounting/all_data_jittered_13425/'
#set path of feature map where you want to store
feature_map_save_path = '/home/akhtar/Naseer2/feature_maps/'
create_dir(feature_map_save_path)

In [4]:
ssnet_maps_path = feature_map_save_path + 'dmaps/'
create_dir(ssnet_maps_path)
densenet_maps_path = feature_map_save_path + 'relu5_blk/'
create_dir(densenet_maps_path)
densenet_gap_maps_path = feature_map_save_path + 'pool5/'
create_dir(densenet_gap_maps_path)
gwap_maps_path = feature_map_save_path + 'gap_maps/'
create_dir(gwap_maps_path)
product_maps_path = feature_map_save_path + 'pf_maps/'
create_dir(product_maps_path)
overlays_path = "{}/overlays/".format(feature_map_save_path)
create_dir(overlays_path)

# Feature Extraction

In [7]:
weights_path = 'densenet121_weights_tf.h5'
model = DenseNet(reduction=0.5, classes=1000, weights_path=weights_path)
model.summary()

hiiii


densenet121.py:49: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (7, 7), strides=(2, 2), use_bias=False, name="conv1")`
  x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
densenet121.py:103: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv2_1_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:113: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv2_1_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:169: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  concat_feat = merge([concat_feat, x], mode='concat', concat_axis=concat_axis, name='concat_'+str(stage)+'_'+str(branch))
/home/akhtar/Naseer/local/lib/pyth

densenet121.py:140: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1), use_bias=False, name="conv3_blk")`
  x = Convolution2D(int(nb_filter * compression), 1, 1, name=conv_name_base, bias=False)(x)
densenet121.py:103: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_1_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:113: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_1_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:103: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_2_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:113: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="

densenet121.py:113: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_21_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:103: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_22_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:113: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_22_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:103: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_23_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:113: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_23_

Instructions for updating:
keep_dims is deprecated, use keepdims instead


densenet121.py:103: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv5_16_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:113: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv5_16_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
data (InputLayer)                (None, 336, 336, 3)   0                                            
____________________________________________________________________________________________________
conv1_zeropadding (ZeroPadding2D (None, 342, 342, 3)   0           data[0][0]                       
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 168, 168, 64)  9408        conv1_zeropadding[0][0]          
____________________________________________________________________________________________________
conv1_bn (BatchNormalization)    (None, 168, 168, 64)  256         conv1[0][0]                      
___________________________________________________________________________________________

In [8]:
image_size_r = 336
image_size_c = 336
def Feature_extract(model,layer_name,feature_name):
    part_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
    part_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    lines = [line.rstrip('\n') for line in open(root_pth + 'filenames.txt')]

    for i in range(0,len(lines)):
        print(i)
        line = lines[i].split(' ')
        image_path = os.path.join(dataset_path,line[0] + '.tiff')
        img = cv2.imread(image_path)
        test_image = np.zeros((1,image_size_r,image_size_c,3))
        test_image[0,:,:,:] = img[:,:,:]


        pth = feature_map_save_path + layer_name
        create_dir(pth)
        save_path = os.path.join(pth, line[0] + '.mat')
        feature_map = part_model.predict(test_image)     
        sio.savemat(save_path, {feature_name : feature_map})
       

In [10]:
layer_name = 'relu5_blk'
feature_name = 'fmap'
Feature_extract(model,layer_name,feature_name)

In [12]:
layer_name = 'pool5'
feature_name = 'fmap'
Feature_extract(model,layer_name,feature_name)

In [13]:
def create_model():
    
    model=Sequential()

    #conv1_1
    model.add(Conv2D(64,kernel_size=3, strides=1,
                 padding='SAME', use_bias=False, 
                 activation='relu',name='conv1_1',batch_input_shape=(None,None,None,3)))
    #conv1_2
    model.add(Conv2D(64, kernel_size=3, strides=1,
                 padding='SAME', use_bias=False, 
                 activation='relu',name='conv1_2'))
    model.add(MaxPooling2D(pool_size=2,strides=2))

    #conv2_1
    model.add(Conv2D(128, kernel_size=3, strides=1,
                 padding='SAME', use_bias=False, 
                 activation='relu',name="conv2_1"))

    #conv2_2
    model.add(Conv2D(128, kernel_size=3, strides=1,
                 padding='SAME', use_bias=False, 
                 activation='relu',name='conv2_2'))
    model.add(MaxPooling2D(pool_size=2,strides=2))

    #conv3_1
    model.add(Conv2D(256, kernel_size=3, strides=1,
                 padding='SAME', use_bias=False,
                 activation='relu',name='conv3_1'))

    #conv3_2
    model.add(Conv2D(256, kernel_size=3, strides=1,
                 padding='SAME', use_bias=False, 
                 activation='relu',name='conv3_2'))

    #conv3_3
    model.add(Conv2D(256, kernel_size=3, strides=1,
                 padding='SAME', use_bias=False, 
                 activation='relu',name='conv3_3'))
    model.add(MaxPooling2D(pool_size=2,strides=2))

    #conv6
    model.add(Conv2D(2048, kernel_size=8, strides=1, use_bias=False,
                 activation='relu',name='conv6'))

    #conv7
    model.add(Conv2D(256, kernel_size=1, strides=1,
                 use_bias=False,
                 activation='relu',name='conv7'))

    #conv8
    model.add(Conv2D(2, kernel_size=1, strides=1,
                 use_bias=False, 
                 name='conv8'))
    
    
    model.add(Activation('softmax'))
#     model.add(GlobalAveragePooling2D())

    
    return model

In [14]:
locations = open(root_pth + 'sh_testing_531_housecounting_data_tagged.txt', 'r').read().split('\n')
locations = [row.split(' ') for row in locations]

save_overlays = True


image_extension = '.tiff'

In [16]:
model = create_model()
model.load_weights('Models/base_weights.hdf5')

img_h = 256
img_w = 256

img_h = np.int(img_h)
img_w = np.int(img_w)

M = [108,104,94] # mean values

for idx,row in enumerate(locations):
    print(idx)
    f = Image.open(dataset_path + row[0] + image_extension)    
    image = np.asarray(f, dtype=np.float32)
    image = cv2.resize(image, (256,256))
    if len(image.shape) == 3:
        image = image.transpose(2, 0, 1)
        
    test_img = np.zeros((1, img_h, img_w, 3))
    overlay = np.zeros((img_h, img_w, 3))
    
    test_img[0,:,:,0] = image[0,:,:] - M[0]
    test_img[0,:,:,1] = image[1,:,:] - M[1] 
    test_img[0,:,:,2] = image[2,:,:] - M[2]
    test_img = np.float32(test_img)
    
    output = model.predict(test_img)
    sample0 = imresize(output[0,:,:,0], [ img_h,  img_w], interp='bilinear', mode='F')
    sample1 = imresize(output[0,:,:,1], [ img_h,  img_w], interp='bilinear', mode='F')         
    
    overlay[:,:,0] = image[0,:,:]
    overlay[:,:,1] = image[1,:,:]
    temp = image[2,:,:]

    vis = (np.double(temp) * np.double(sample0)) + np.double(255*(sample1))
    overlay[:,:,2] =  vis

    filename = "{}/{}.npy".format(feature_map_save_path + 'dmaps', row[0])
    sio.savemat(filename , {'dmap':sample1})
    
    if save_overlays:
        overlay_path = "{}/{}.png".format(overlays_path, row[0])
        imsave(overlay_path, overlay)
    


Extract gap_maps and pf_maps

In [23]:
locations = open(root_pth +'filenames.txt', 'r').read().split('\n')
locations = [row.split(' ') for row in locations]

In [18]:

def GAP(feature_map):
    return np.average(feature_map, axis = (0, 1))
mm=0
for row in locations:
    name = row[0]
    #ssnet_map = np.load(open(ssnet_maps_path + name + '.npy', 'rb'))

    ssnet_map = sio.loadmat(ssnet_maps_path + name + '_1.mat')
    ssnet_map = ssnet_map['Prob']


    #densenet_map = np.load(open(densenet_maps_path + name + '.npy', 'rb'))
    densenet_map =  sio.loadmat(densenet_maps_path + name + '.mat')
    densenet_map = densenet_map['fmap']
    densenet_map = densenet_map[0, :, :, :]
    densenet_gap_map = GAP(densenet_map)
    
    
    resized_densenet_map = np.zeros((42, 42, 1024))
    for i in range(1024):
        resized_densenet_map[:, :, i] = imresize(densenet_map[:, :, i], (42, 42), interp = "bilinear", mode = 'F')
    
    resized_ssnet_map = np.zeros((42, 42))
    #resized_ssnet_map = imresize(ssnet_map, (42, 42), interp = "bilinear", mode = 'F')
    resized_ssnet_map = cv2.resize(ssnet_map, dsize=(42, 42), interpolation=cv2.INTER_LINEAR)
    
    product_map = np.zeros((42, 42, 1024))
    
    for i in range(1024):
        product_map[:, :, i] = np.multiply(resized_densenet_map[:, :, i], resized_ssnet_map)
    product_map_gap = GAP(product_map)
    
    #sio.savemat(densenet_gap_maps_path + name, {'pgap':densenet_gap_map})
    sio.savemat(product_maps_path + name, {'pfmap':product_map})
    sio.savemat(gwap_maps_path + name, {'pgap':product_map_gap})
    print(mm)
    mm = mm+1
    
    #np.save(open(densenet_gap_maps_path + name + '.npy', 'wb'), densenet_gap_map)
    #np.save(open(product_maps_path + name + '.npy', 'wb'), product_map)
    #np.save(open(gwap_maps_path + name + '.npy', 'wb'), product_map_gap)'''

# FusionNet 

In [5]:
def generator(train_indices, batch_size, itr, lines, image_size=336):
    # Create empty arrays to contain batch of features and labels#
    batch_images = []
    batch_labels = []
    batch_pmap = []
    batch_product = []
    startInd = itr*batch_size
    endInd = startInd + batch_size
    p_mapsize = 32
    fmap_size = 42
    for i in range(startInd, endInd):
        line = lines[i].split(' ')
#         image_path = os.path.join('/media/itu/CV_Lab2/Dr_waqas', 'DenseNetFeatures_pool5', line[0] + '.mat')
        image_path = os.path.join(feature_map_save_path+ 'pool5', line[0] + '.mat')
        #print(image_path)
        fmap1 = sio.loadmat(image_path)
        fmap1 = fmap1['fmap']
        fmap = fmap1.ravel()
        
#         image_path1 = os.path.join('/media/itu/CV_Lab2/Dr_waqas', 'DenseNetFeatures_Pmaps1only_GAP', line[0] + '.mat')
        image_path1 = os.path.join(feature_map_save_path+'gap_maps', line[0] + '.mat')
        #print(image_path)
        fmap11 = sio.loadmat(image_path1)
        fmap11 = fmap11['pgap']
        fmap1 = fmap11.ravel()
        
#         image_path2 = os.path.join('/media/itu/CV_Lab2/Dr_waqas', 'DenseNetFeatures_product', line[0] + '.mat')
        image_path2 = os.path.join(feature_map_save_path + 'pf_maps', line[0] + '.mat')
        #print(image_path)
        fmap22 = sio.loadmat(image_path2)
        fmap22 = fmap22['pfmap']
        fmap2 = np.zeros((fmap_size,fmap_size,1024))
        fmap2[:,:,:] = fmap22[:,:,:]
        
        
        batch_images.append(fmap)
        batch_pmap.append(fmap1)
        batch_product.append(fmap2)
        batch_labels.append(np.int(line[1]))
        
    batch_images = np.array(batch_images, dtype=np.float32)
    batch_pmap = np.array(batch_pmap, dtype=np.float32)
    batch_product = np.array(batch_product, dtype=np.float32)
    batch_labels = np.array(batch_labels, dtype=np.float32)
    
    
    return batch_images, batch_pmap, batch_product, batch_labels

In [6]:
def root_mean_squared_error(y_true, y_pred):
    """
    RMSE loss function
    """
    
    loss = K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))
#     loss = K.clip(loss,0,5)
    return  loss

In [7]:
def create_model(lr_r):
  
    f_map = Input(shape=(1024,), name='DenseNet')
    p_map = Input(shape=(1024,), name='SSNEt')
    prod_map = Input(shape=(42,42,1024), name='product')
    
    x_d = Dense(512, init='glorot_normal')(f_map)
    x_d = LeakyReLU(alpha=0.3)(x_d)
    x_d = Dropout(0.5)(x_d)
    
    x_s = Dense(512, init='glorot_normal')(p_map)
    x_s = LeakyReLU(alpha=0.3)(x_s)
    x_s = Dropout(0.5)(x_s)
    
    x_p = Convolution2D(1, (1, 1), init='glorot_normal', W_regularizer=l1(0.001))(prod_map)
    x_p = LeakyReLU(alpha=0.3)(x_p)
    x_p = Flatten()(x_p)
    x_p = Dense(512, init='glorot_normal')(x_p)
    x_p = LeakyReLU(alpha=0.3)(x_p)
    x_p = Dropout(0.5)(x_p)
    
    
    x = Concatenate(axis=-1)([x_d,  x_s, x_p]) 
    x = Dense(512, init='glorot_normal')(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Dropout(0.5)(x)
    x = Dense(32, init='glorot_normal')(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Dropout(0.5)(x)
    x = Dense(1, init='glorot_normal')(x)    
    x = Activation('linear')(x)
    
    adagrad=Adagrad(lr=lr_r, epsilon=1e-08)
    
    model_finetune = Model(inputs=[f_map, p_map, prod_map], outputs=x)    
    model_finetune.compile(loss=root_mean_squared_error, optimizer=adagrad, metrics=['accuracy'])
#     model_finetune.compile(loss='mean_squared_error', optimizer=adagrad, metrics=['accuracy'])
   
    return model_finetune

In [8]:
lr_r = 0.01
model = create_model(lr_r)
model.summary()

/home/akhtar/Naseer/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, kernel_initializer="glorot_normal")`
  import sys
/home/akhtar/Naseer/lib/python2.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, kernel_initializer="glorot_normal")`
  # This is added back by InteractiveShellApp.init_path()
/home/akhtar/Naseer/lib/python2.7/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (1, 1), kernel_regularizer=<keras.reg..., kernel_initializer="glorot_normal")`
  from ipykernel import kernelapp as app


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
product (InputLayer)             (None, 42, 42, 1024)  0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 42, 42, 1)     1025        product[0][0]                    
____________________________________________________________________________________________________
leaky_re_lu_3 (LeakyReLU)        (None, 42, 42, 1)     0           conv2d_1[0][0]                   
__________________________________________________________________________

/home/akhtar/Naseer/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, kernel_initializer="glorot_normal")`
/home/akhtar/Naseer/lib/python2.7/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, kernel_initializer="glorot_normal")`
/home/akhtar/Naseer/lib/python2.7/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, kernel_initializer="glorot_normal")`
/home/akhtar/Naseer/lib/python2.7/site-packages/ipykernel_launcher.py:30: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="glorot_normal")`


In [28]:
model_weight_path = root_pth  + 'FusionNet_weights'
create_dir(model_weight_path)

In [29]:
model.load_weights(root_pth+'FusionNet_weights/weights_24_Epochs.hdf5')

Training 

In [ ]:
batch_size = 1
nb_epoch = 3000

#model = create_model()
# model.load_weights('5_EXPERIMNTS_FOR_PAPER/exp8_FUSION/HouseDetection_densenet_3fusion_208_Epochs.hdf5')
lines = [line.rstrip('\n') for line in open(root_pth +'sh_training_jitter_10755_housecounting_data_tagged.txt')]

indices = np.arange(len(lines))
train_indices = indices[:]    #split training
print(train_indices.shape)
iterations_t = np.int(len(train_indices)/batch_size)
t_loss_ep = []

for epoch in range(nb_epoch):
    ## training
    t_loss = []
    start = time.time()
    for i in range(iterations_t):
        
        X_train, X_pmap, X_prod, y_train = generator(train_indices, batch_size, i, lines, image_size=336)  
        history = model.fit([X_train, X_pmap, X_prod], y_train, verbose=0)
        t_loss.append(history.history['loss'])
        if(i % 100 == 0):
            it_loss = np.mean(np.array(t_loss, dtype=np.float32))
            print("Epoch: {} itrerations: {} Training Loss: {}".format(epoch,i, it_loss) )
    stop = time.time()
    duration = stop - start
    t_loss_ep.append(np.mean(np.array(t_loss, dtype=np.float32)))
#     print("################################MAIN LOSS#############################")
#     print("Epoch: {}  Training Loss: {}  Time: {} sec".format(epoch+209, t_loss_ep[epoch], duration) )
    
            
    model_name = model_weight_path  + '/weights_' + str(epoch) + "_Epochs.hdf5"
    model.save_weights(model_name)
    


(10755,)
Epoch: 0 itrerations: 0 Training Loss: 4.28543615341
Epoch: 0 itrerations: 100 Training Loss: 15.3545198441
Epoch: 0 itrerations: 200 Training Loss: 12.1068267822
Epoch: 0 itrerations: 300 Training Loss: 10.312046051
Epoch: 0 itrerations: 400 Training Loss: 9.81051254272
Epoch: 0 itrerations: 500 Training Loss: 8.7645483017
Epoch: 0 itrerations: 600 Training Loss: 8.90727710724
Epoch: 0 itrerations: 700 Training Loss: 8.68954753876
Epoch: 0 itrerations: 800 Training Loss: 8.69816493988
Epoch: 0 itrerations: 900 Training Loss: 8.48792076111
Epoch: 0 itrerations: 1000 Training Loss: 8.30631542206
Epoch: 0 itrerations: 1100 Training Loss: 8.14824581146
Epoch: 0 itrerations: 1200 Training Loss: 8.25808906555
Epoch: 0 itrerations: 1300 Training Loss: 8.05711174011
Epoch: 0 itrerations: 1400 Training Loss: 8.11654758453
Epoch: 0 itrerations: 1500 Training Loss: 8.018866539
Epoch: 0 itrerations: 1600 Training Loss: 7.95171833038
Epoch: 0 itrerations: 1700 Training Loss: 7.86840629578

Epoch: 1 itrerations: 3900 Training Loss: 5.25096702576
Epoch: 1 itrerations: 4000 Training Loss: 5.25128078461
Epoch: 1 itrerations: 4100 Training Loss: 5.23433589935
Epoch: 1 itrerations: 4200 Training Loss: 5.25011253357
Epoch: 1 itrerations: 4300 Training Loss: 5.26156711578
Epoch: 1 itrerations: 4400 Training Loss: 5.27962398529
Epoch: 1 itrerations: 4500 Training Loss: 5.28161716461
Epoch: 1 itrerations: 4600 Training Loss: 5.291036129
Epoch: 1 itrerations: 4700 Training Loss: 5.2882065773
Epoch: 1 itrerations: 4800 Training Loss: 5.28749227524
Epoch: 1 itrerations: 4900 Training Loss: 5.26590061188
Epoch: 1 itrerations: 5000 Training Loss: 5.25592374802
Epoch: 1 itrerations: 5100 Training Loss: 5.25083208084
Epoch: 1 itrerations: 5200 Training Loss: 5.25567054749
Epoch: 1 itrerations: 5300 Training Loss: 5.23260593414
Epoch: 1 itrerations: 5400 Training Loss: 5.24987983704
Epoch: 1 itrerations: 5500 Training Loss: 5.23260736465
Epoch: 1 itrerations: 5600 Training Loss: 5.2327342

Epoch: 2 itrerations: 7800 Training Loss: 4.99193000793
Epoch: 2 itrerations: 7900 Training Loss: 4.9998998642
Epoch: 2 itrerations: 8000 Training Loss: 4.99886846542
Epoch: 2 itrerations: 8100 Training Loss: 4.98927974701
Epoch: 2 itrerations: 8200 Training Loss: 4.99568557739
Epoch: 2 itrerations: 8300 Training Loss: 4.98779153824
Epoch: 2 itrerations: 8400 Training Loss: 4.98696708679
Epoch: 2 itrerations: 8500 Training Loss: 4.98662519455
Epoch: 2 itrerations: 8600 Training Loss: 4.98356723785
Epoch: 2 itrerations: 8700 Training Loss: 4.98570728302
Epoch: 2 itrerations: 8800 Training Loss: 4.98607587814
Epoch: 2 itrerations: 8900 Training Loss: 4.98292589188
Epoch: 2 itrerations: 9000 Training Loss: 4.98847103119
Epoch: 2 itrerations: 9100 Training Loss: 4.98972511292
Epoch: 2 itrerations: 9200 Training Loss: 4.99161434174
Epoch: 2 itrerations: 9300 Training Loss: 4.98361968994
Epoch: 2 itrerations: 9400 Training Loss: 4.97889566422
Epoch: 2 itrerations: 9500 Training Loss: 4.97629

Epoch: 4 itrerations: 900 Training Loss: 4.67462921143
Epoch: 4 itrerations: 1000 Training Loss: 4.65913486481
Epoch: 4 itrerations: 1100 Training Loss: 4.61375617981
Epoch: 4 itrerations: 1200 Training Loss: 4.72556304932
Epoch: 4 itrerations: 1300 Training Loss: 4.7138671875
Epoch: 4 itrerations: 1400 Training Loss: 4.77700138092
Epoch: 4 itrerations: 1500 Training Loss: 4.76285219193
Epoch: 4 itrerations: 1600 Training Loss: 4.75558710098
Epoch: 4 itrerations: 1700 Training Loss: 4.74680757523
Epoch: 4 itrerations: 1800 Training Loss: 4.74588489532
Epoch: 4 itrerations: 1900 Training Loss: 4.7876496315
Epoch: 4 itrerations: 2000 Training Loss: 4.81170368195
Epoch: 4 itrerations: 2100 Training Loss: 4.78027582169
Epoch: 4 itrerations: 2200 Training Loss: 4.79221343994
Epoch: 4 itrerations: 2300 Training Loss: 4.82284975052
Epoch: 4 itrerations: 2400 Training Loss: 4.79176330566
Epoch: 4 itrerations: 2500 Training Loss: 4.76947402954
Epoch: 4 itrerations: 2600 Training Loss: 4.7571897

Epoch: 5 itrerations: 4800 Training Loss: 4.90960550308
Epoch: 5 itrerations: 4900 Training Loss: 4.90338850021
Epoch: 5 itrerations: 5000 Training Loss: 4.87696743011
Epoch: 5 itrerations: 5100 Training Loss: 4.86368989944
Epoch: 5 itrerations: 5200 Training Loss: 4.87488508224
Epoch: 5 itrerations: 5300 Training Loss: 4.87199258804
Epoch: 5 itrerations: 5400 Training Loss: 4.8784160614
Epoch: 5 itrerations: 5500 Training Loss: 4.87297010422
Epoch: 5 itrerations: 5600 Training Loss: 4.86943244934
Epoch: 5 itrerations: 5700 Training Loss: 4.8657913208
Epoch: 5 itrerations: 5800 Training Loss: 4.86817884445
Epoch: 5 itrerations: 5900 Training Loss: 4.8650932312
Epoch: 5 itrerations: 6000 Training Loss: 4.86750555038
Epoch: 5 itrerations: 6100 Training Loss: 4.86302471161
Epoch: 5 itrerations: 6200 Training Loss: 4.87208986282
Epoch: 5 itrerations: 6300 Training Loss: 4.87150239944
Epoch: 5 itrerations: 6400 Training Loss: 4.8614449501
Epoch: 5 itrerations: 6500 Training Loss: 4.85844516

...

Epoch: 45 itrerations: 8200 Training Loss: 4.30957269669
Epoch: 45 itrerations: 8300 Training Loss: 4.30896806717
Epoch: 45 itrerations: 8400 Training Loss: 4.31337594986
Epoch: 45 itrerations: 8500 Training Loss: 4.31849479675
Epoch: 45 itrerations: 8600 Training Loss: 4.3117146492
Epoch: 45 itrerations: 8700 Training Loss: 4.31057929993
Epoch: 45 itrerations: 8800 Training Loss: 4.31127166748
Epoch: 45 itrerations: 8900 Training Loss: 4.3107380867
Epoch: 45 itrerations: 9000 Training Loss: 4.31261730194
Epoch: 45 itrerations: 9100 Training Loss: 4.30514907837
Epoch: 45 itrerations: 9200 Training Loss: 4.31019496918
Epoch: 45 itrerations: 9300 Training Loss: 4.31150817871
Epoch: 45 itrerations: 9400 Training Loss: 4.30557823181
Epoch: 45 itrerations: 9500 Training Loss: 4.30525016785
Epoch: 45 itrerations: 9600 Training Loss: 4.31839704514
Epoch: 45 itrerations: 9700 Training Loss: 4.32386064529
Epoch: 45 itrerations: 9800 Training Loss: 4.31919193268
Epoch: 45 itrerations: 9900 Train

Epoch: 47 itrerations: 1100 Training Loss: 4.27560043335
Epoch: 47 itrerations: 1200 Training Loss: 4.37273550034
Epoch: 47 itrerations: 1300 Training Loss: 4.3814330101
Epoch: 47 itrerations: 1400 Training Loss: 4.45303916931
Epoch: 47 itrerations: 1500 Training Loss: 4.44438886642
Epoch: 47 itrerations: 1600 Training Loss: 4.47270774841
Epoch: 47 itrerations: 1700 Training Loss: 4.49856901169
Epoch: 47 itrerations: 1800 Training Loss: 4.50646781921
Epoch: 47 itrerations: 1900 Training Loss: 4.49980974197
Epoch: 47 itrerations: 2000 Training Loss: 4.49943399429
Epoch: 47 itrerations: 2100 Training Loss: 4.48630094528
Epoch: 47 itrerations: 2200 Training Loss: 4.45872163773
Epoch: 47 itrerations: 2300 Training Loss: 4.4588136673
Epoch: 47 itrerations: 2400 Training Loss: 4.42151117325
Epoch: 47 itrerations: 2500 Training Loss: 4.37255907059
Epoch: 47 itrerations: 2600 Training Loss: 4.36536884308
Epoch: 47 itrerations: 2700 Training Loss: 4.38435935974
Epoch: 47 itrerations: 2800 Train

Epoch: 48 itrerations: 4800 Training Loss: 4.36036682129
Epoch: 48 itrerations: 4900 Training Loss: 4.34064054489
Epoch: 48 itrerations: 5000 Training Loss: 4.31964111328
Epoch: 48 itrerations: 5100 Training Loss: 4.30995798111
Epoch: 48 itrerations: 5200 Training Loss: 4.30075979233
Epoch: 48 itrerations: 5300 Training Loss: 4.30391836166
Epoch: 48 itrerations: 5400 Training Loss: 4.3105134964
Epoch: 48 itrerations: 5500 Training Loss: 4.30820989609
Epoch: 48 itrerations: 5600 Training Loss: 4.3239274025
Epoch: 48 itrerations: 5700 Training Loss: 4.32197713852
Epoch: 48 itrerations: 5800 Training Loss: 4.33116006851
Epoch: 48 itrerations: 5900 Training Loss: 4.33965158463
Epoch: 48 itrerations: 6000 Training Loss: 4.34210634232
Epoch: 48 itrerations: 6100 Training Loss: 4.32422971725
Epoch: 48 itrerations: 6200 Training Loss: 4.33617687225
Epoch: 48 itrerations: 6300 Training Loss: 4.33964824677
Epoch: 48 itrerations: 6400 Training Loss: 4.3369641304
Epoch: 48 itrerations: 6500 Traini

Epoch: 49 itrerations: 8500 Training Loss: 4.36025428772
Epoch: 49 itrerations: 8600 Training Loss: 4.35394668579
Epoch: 49 itrerations: 8700 Training Loss: 4.35061264038
Epoch: 49 itrerations: 8800 Training Loss: 4.35182857513
Epoch: 49 itrerations: 8900 Training Loss: 4.35116291046
Epoch: 49 itrerations: 9000 Training Loss: 4.35654020309
Epoch: 49 itrerations: 9100 Training Loss: 4.3563914299
Epoch: 49 itrerations: 9200 Training Loss: 4.35610151291
Epoch: 49 itrerations: 9300 Training Loss: 4.3469991684
Epoch: 49 itrerations: 9400 Training Loss: 4.33788490295
Epoch: 49 itrerations: 9500 Training Loss: 4.3369550705
Epoch: 49 itrerations: 9600 Training Loss: 4.349568367
Epoch: 49 itrerations: 9700 Training Loss: 4.35305404663
Epoch: 49 itrerations: 9800 Training Loss: 4.34988451004
Epoch: 49 itrerations: 9900 Training Loss: 4.34784412384
Epoch: 49 itrerations: 10000 Training Loss: 4.36517810822
Epoch: 49 itrerations: 10100 Training Loss: 4.36897802353
Epoch: 49 itrerations: 10200 Train

Epoch: 51 itrerations: 1400 Training Loss: 4.49801492691
Epoch: 51 itrerations: 1500 Training Loss: 4.45982170105
Epoch: 51 itrerations: 1600 Training Loss: 4.50043535233
Epoch: 51 itrerations: 1700 Training Loss: 4.48444747925
Epoch: 51 itrerations: 1800 Training Loss: 4.48810720444
Epoch: 51 itrerations: 1900 Training Loss: 4.52190732956
Epoch: 51 itrerations: 2000 Training Loss: 4.52839851379
Epoch: 51 itrerations: 2100 Training Loss: 4.46641111374
Epoch: 51 itrerations: 2200 Training Loss: 4.44138050079
Epoch: 51 itrerations: 2300 Training Loss: 4.42792701721
Epoch: 51 itrerations: 2400 Training Loss: 4.38891983032
Epoch: 51 itrerations: 2500 Training Loss: 4.39550828934
Epoch: 51 itrerations: 2600 Training Loss: 4.37844657898
Epoch: 51 itrerations: 2700 Training Loss: 4.41119289398
Epoch: 51 itrerations: 2800 Training Loss: 4.39333248138
Epoch: 51 itrerations: 2900 Training Loss: 4.35811328888
Epoch: 51 itrerations: 3000 Training Loss: 4.40549564362
Epoch: 51 itrerations: 3100 Tra

Testing 

In [11]:
lines = [line.rstrip('\n') for line in open(root_pth +'sh_testing_531_housecounting_data_tagged.txt')]


model.load_weights(root_pth+'FusionNet_weights/weights_45_Epochs.hdf5')

data = ""
data_low = ""
data_low_gt = ""
data_high = ""
data_high_gt = ""
data_medium = ""
data_medium_gt = ""

mean_abs_error = 0
mean_sqrd_error = 0

mean_abs_error_low = 0
mean_sqrd_error_low = 0

mean_abs_error_high = 0
mean_sqrd_error_high = 0

mean_abs_error_medium = 0
mean_sqrd_error_medium = 0

total_data = len(lines)
count = 0
count_low = 0
count_high = 0
count_medium = 0
fmap_size = 42
p_mapsize = 32

print(len(lines))
for i in range(len(lines)):
    line = lines[i].split(' ')
    gt_count = np.int(line[1])
    #print(lines[i])
    
#     image_path = os.path.join('/media/itu/CV_Lab2/Dr_waqas', 'DenseNetFeatures_pool5', line[0] + '.mat')
    image_path = os.path.join(feature_map_save_path + 'pool5', line[0] + '.mat')
    #print(image_path)
    fmap1 = sio.loadmat(image_path)
    fmap1 = fmap1['fmap']
    fmap = fmap1.ravel()
    test_image = np.zeros((1,1024))
    test_image[0,:] = fmap

#     image_path1 = os.path.join('/media/itu/CV_Lab2/Dr_waqas', 'DenseNetFeatures_Pmaps1only_GAP', line[0] + '.mat')
    image_path1 = os.path.join(feature_map_save_path + 'gap_maps', line[0] + '.mat')
    #print(image_path)
    fmap11 = sio.loadmat(image_path1)
    fmap11 = fmap11['pgap']
    fmap1 = fmap11.ravel()
    test_pmap = np.zeros((1,1024))
    test_pmap[0, :] = fmap1
    
#     image_path2 = os.path.join('/media/itu/CV_Lab2/Dr_waqas', 'DenseNetFeatures_product', line[0] + '.mat')
    image_path2 = os.path.join(feature_map_save_path + 'pf_maps', line[0] + '.mat')
    #print(image_path)
    fmap22 = sio.loadmat(image_path2)
#     print(fmap22)
    fmap22 = fmap22['pfmap']
    fmap2 = np.zeros((fmap_size,fmap_size,1024))
    fmap2[:,:,:] = fmap22[:,:,:]
    test_product = np.zeros((1,42,42,1024))
    test_product[0, :, :, :] = fmap2
         
    
    output = model.predict([test_image, test_pmap, test_product])
    if output < 0:
        output = 0
    print([gt_count, output])
    
    abs_error = np.abs(gt_count - output)
    mean_abs_error = mean_abs_error + abs_error
    
    sqrd_error = (gt_count - output) * (gt_count - output)
    mean_sqrd_error = mean_sqrd_error + sqrd_error  
    
    if gt_count >= 0 and gt_count <= 30:
            count_low = count_low + 1
            mean_abs_error_low = mean_abs_error_low + abs_error
            mean_sqrd_error_low = mean_sqrd_error_low + sqrd_error
            data_low = data_low + line[0] + " " + str(float(output)) + "\n"
            data_low_gt = data_low_gt + line[0] + " " + str(float(gt_count)) + "\n"
        
    if gt_count >= 61 and gt_count <= 100:
            count_high = count_high + 1
            mean_abs_error_high = mean_abs_error_high + abs_error
            mean_sqrd_error_high = mean_sqrd_error_high + sqrd_error
            data_high = data_high + line[0] + " " + str(float(output)) + "\n"
            data_high_gt = data_high_gt + line[0] + " " + str(float(gt_count)) + "\n"
            
    if gt_count >= 31 and gt_count <= 60:
            count_medium = count_medium + 1
            mean_abs_error_medium = mean_abs_error_medium + abs_error
            mean_sqrd_error_medium = mean_sqrd_error_medium + sqrd_error
            data_medium = data_medium + line[0] + " " + str(float(output)) + "\n"
            data_medium_gt = data_medium_gt + line[0] + " " + str(float(gt_count)) + "\n"
    
    
    data = data + line[0] + " " + str(float(output)) + "\n"
f = open(root_pth + 'HouseDetection_densenet_3fusion_total.txt','w')
f.write(data)

f_mean_abs_error = mean_abs_error / total_data
f_mean_sqrd_error = mean_sqrd_error / total_data

f_mean_abs_error_low = mean_abs_error_low / count_low
f_mean_sqrd_error_low = mean_sqrd_error_low / count_low

f_mean_abs_error_high = mean_abs_error_high / count_high
f_mean_sqrd_error_high = mean_sqrd_error_high / count_high

f_mean_abs_error_medium = mean_abs_error_medium / count_medium
f_mean_sqrd_error_medium = mean_sqrd_error_medium / count_medium

print('##################################### ERROR VALUES ###########################################')
print("Mean Squared Error: {} ".format(f_mean_sqrd_error))
print("Mean Absolute Error: {} ".format(f_mean_abs_error))
print("Number of Instances: {} ".format(total_data))
print('##############################################################################################')

print('##################################### ERROR VALUES-LOW ###########################################')
print("Mean Squared Error: {} ".format(f_mean_sqrd_error_low))
print("Mean Absolute Error: {} ".format(f_mean_abs_error_low))
print("Number of Instances: {} ".format(count_low))
print('##############################################################################################')

print('##################################### ERROR VALUES-MEDIUM ###########################################')
print("Mean Squared Error: {} ".format(f_mean_sqrd_error_medium))
print("Mean Absolute Error: {} ".format(f_mean_abs_error_medium))
print("Number of Instances: {} ".format(count_medium))
print('##############################################################################################')

print('##################################### ERROR VALUES-HIGH ###########################################')
print("Mean Squared Error: {} ".format(f_mean_sqrd_error_high))
print("Mean Absolute Error: {} ".format(f_mean_abs_error_high))
print("Number of Instances: {} ".format(count_high))
print('##############################################################################################')

531
[0, 0]
[5, array([[4.196158]], dtype=float32)]
[1, array([[0.24875116]], dtype=float32)]
[40, array([[40.92314]], dtype=float32)]
[30, array([[33.747246]], dtype=float32)]
[4, array([[7.877047]], dtype=float32)]
[0, array([[0.22600794]], dtype=float32)]
[0, array([[0.18292284]], dtype=float32)]
[47, array([[36.541603]], dtype=float32)]
[16, array([[17.61943]], dtype=float32)]
[15, array([[14.413607]], dtype=float32)]
[12, array([[10.24546]], dtype=float32)]
[1, array([[1.08038]], dtype=float32)]
[0, array([[0.44815826]], dtype=float32)]
[15, array([[10.805791]], dtype=float32)]
[22, array([[27.896841]], dtype=float32)]
[36, array([[34.75606]], dtype=float32)]
[1, array([[5.4089055]], dtype=float32)]
[0, array([[0.9697712]], dtype=float32)]
[0, array([[0.3055945]], dtype=float32)]
[15, array([[12.452367]], dtype=float32)]
[0, array([[0.16928798]], dtype=float32)]
[0, array([[0.20835727]], dtype=float32)]
[0, 0]
[0, array([[0.19815922]], dtype=float32)]
[49, array([[36.859303]], dtyp

[0, array([[0.185565]], dtype=float32)]
[24, array([[25.50395]], dtype=float32)]
[9, array([[9.305759]], dtype=float32)]
[31, array([[31.455713]], dtype=float32)]
[21, array([[23.389883]], dtype=float32)]
[22, array([[19.179007]], dtype=float32)]
[0, array([[0.38897687]], dtype=float32)]
[0, array([[0.5891069]], dtype=float32)]
[26, array([[20.337294]], dtype=float32)]
[44, array([[45.36332]], dtype=float32)]
[7, array([[8.320624]], dtype=float32)]
[11, array([[7.1933765]], dtype=float32)]
[31, array([[24.627613]], dtype=float32)]
[6, array([[7.2368193]], dtype=float32)]
[33, array([[32.44287]], dtype=float32)]
[9, array([[11.084945]], dtype=float32)]
[30, array([[28.87394]], dtype=float32)]
[4, array([[8.205449]], dtype=float32)]
[17, array([[18.195955]], dtype=float32)]
[0, array([[0.9292843]], dtype=float32)]
[20, array([[18.069508]], dtype=float32)]
[0, array([[0.10639477]], dtype=float32)]
[0, array([[0.37319398]], dtype=float32)]
[60, array([[34.8203]], dtype=float32)]
[0, array(

[70, array([[72.24823]], dtype=float32)]
[14, array([[13.961662]], dtype=float32)]
[0, array([[0.54059976]], dtype=float32)]
[16, array([[20.421652]], dtype=float32)]
[0, array([[0.37325704]], dtype=float32)]
[16, array([[9.085379]], dtype=float32)]
[0, array([[0.39915645]], dtype=float32)]
[0, array([[0.05248296]], dtype=float32)]
[21, array([[18.738104]], dtype=float32)]
[32, array([[34.220856]], dtype=float32)]
[5, array([[2.9084468]], dtype=float32)]
[7, array([[6.279581]], dtype=float32)]
[0, array([[0.32987416]], dtype=float32)]
[0, array([[1.3330953]], dtype=float32)]
[0, array([[0.06690645]], dtype=float32)]
[12, array([[13.01031]], dtype=float32)]
[0, 0]
[16, array([[13.269768]], dtype=float32)]
[31, array([[26.292076]], dtype=float32)]
[10, array([[6.495773]], dtype=float32)]
[0, array([[0.40487373]], dtype=float32)]
[0, array([[0.13923007]], dtype=float32)]
[0, array([[0.7688144]], dtype=float32)]
[1, array([[2.3446767]], dtype=float32)]
[0, array([[0.07812965]], dtype=float